In [39]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
import copy
from DS import Queue, PriorityQueue, Stack

### Problem

In the 8-puzzle, you are given a $3 \times 3$ board where each number 1 through 8 occupies a $1 \times 1$ square. Not that there are 9 possible $1 \times 1$ squares on the $3 \times 3$ board; consequently, there is always one space that is free. Any of the squares adjacent to the free square can "slide" into the free square. Sliding a square into the free space is considered an action. Given the board in a particular state, we want to take a series of actions that would get us the board in its solved state (shown below).

![text](solved.png)

This problem can be solved using graph-traversal algorithms. The state would encode the locations of numbers on the board, and the edges constitute actions (i.e. moves)

First, let us create a class that would help us encapsulate the state of the board in a way that makes a graph traversal easier

In [144]:
def manhattan_distance(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

class State(object):
    def __init__(self, board):
        self.positions = {}
        self.board = board
        self.free = None
        for i in range(len(board)):
            for j in range(len(board)):
                if board[i][j] == None:
                    self.free = (i, j)
                else:
                    self.positions[board[i][j]] = (i, j)
        self.f = 0 # approx cost from start to goal through this state
        self.g = 0 # actual cost from start
        self.h = 0 # approx cost from goal
    
    # used to test when we reach the goal state, amongst other things
    def distance_from(self, other):
        total = 0
        for elem, p1 in self.positions.items():
            p2 = other.positions[elem]
            total += manhattan_distance(p1, p2)
        return total
    
    # give string representation for easier display
    def __str__(self):
        content = ''
        for i in range(3):
            for j in range(3):
                elem = str(self.board[i][j]) if self.board[i][j] is not None else '*'
                content +=  elem + ' '
            content += '\n'
        return content
    
    # used to provide hashcode for hashing-based data structures like dictionaries and sets
    def __hash__(self):
        temp = [tuple(x) for x in self.board]
        temp = tuple(temp)
        return hash(temp)
    
    # need to give dummy implementation for priority queue based algorithms
    def __lt__(self, other):
        return 0

In [145]:

# helper function to compute new board from old board
def swap_on_board(board, pos1, pos2):
    ans = copy.deepcopy(board)
    x_len = len(ans)
    y_len = len(ans)
    temp = ans[pos1[0]][pos1[1]]
    ans[pos1[0]][pos1[1]] = ans[pos2[0]][pos2[1]]
    ans[pos2[0]][pos2[1]] = temp
    return ans
        


class Graph(object):
    def __init__(self):
        self.solution_state = State(
            [[1, 2, 3],
            [4, 5, 6],
            [7, 8, None]]
        )
        
    # admissable heuristic on states
    def h(self, state):
        return state.distance_from(self.solution_state)
    
    # generate states that can be reached from another state 
    def get_edges(self, state):
        x, y = state.free 
        movements = [(-1, 0), (0, 1), (0, -1), (1, 0)] # can swap left, up, down, or right
        for x_diff, y_diff in movements:
            x_prime = x_diff + x # compute swap positions
            y_prime = y_diff + y
            if x_prime > -1 and x_prime < 3 and y_prime > -1 and y_prime < 3: # if swap position is in the board's boundaries
                pos1 = (x_prime, y_prime)
                pos2 = state.free
                adj_state = State(swap_on_board(state.board, pos1, pos2)) # create new state for neighbour
                yield adj_state, 1 # yield both adjacent state and cost from current state to adjacent state
                        
                

In [146]:
graph = Graph()

In [147]:
print(graph.solution_state)
for s, i in graph.get_edges(graph.solution_state):
    print(s)

1 2 3 
4 5 6 
7 8 * 

1 2 3 
4 5 * 
7 8 6 

1 2 3 
4 5 6 
7 * 8 



In [159]:
def print_solution(parents, goal):
    curr = goal
    stack = Stack()
    stack.push(curr)
    while parents[curr]:
        curr = parents[curr]
        stack.push(curr)
    while stack:
        print(stack.pop())
        
def bfs(start, goal_test, graph):
    fronteir = Queue()
    fronteir.enqueue(start)
    explored = set()
    parents = {start: None}
    while fronteir:
        current = fronteir.dequeue()
        explored.add(current)
        if goal_test(current):
            print_solution(parents, current)
            return True
        for adj_state, cost in graph.get_edges(current):
            if adj_state not in fronteir and adj_state not in explored:
                fronteir.enqueue(adj_state)
                parents[adj_state] = current
    print('No Solution found')
    return False
        
    

def a_star(start, goal_test, graph):
    pqueue = PriorityQueue()
    pqueue.push(0, start)
    parents = {start: None}
    explored = set()
    while pqueue:
        i, current = pqueue.pop()
        explored.add(current)
        if goal_test(current):
            print_solution(parents, current)
            return True
        for adj_state, cost in graph.get_edges(current):
            g = current.g + cost
            h = graph.h(adj_state)
            f = g + h
            if (adj_state in pqueue and pqueue[adj_state] > f) or adj_state not in pqueue:
                adj_state.f = f
                adj_state.g = g
                adj_state.h = h
                pqueue[adj_state] = f
                parents[adj_state] = current
    print('No Solution found')
    return False
            
            
        

In [160]:
def goal_test(state):
    solution_state = State(
            [[1, 2, 3],
            [4, 5, 6],
            [7, 8, None]]
        )
    return solution_state.distance_from(state) == 0

start_state = State(
    [
        [1, 2, 3],
        [None, 4, 6],
        [7, 5, 8]
    ]
)

In [161]:
bfs(start_state, goal_test, graph)

1 2 3 
* 4 6 
7 5 8 

1 2 3 
4 * 6 
7 5 8 

1 2 3 
4 5 6 
7 * 8 

1 2 3 
4 5 6 
7 8 * 



True